In [ ]:
# # from BM25Vectorizer import Bm25Vectorizerc
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.utils.validation import check_is_fitted, check_array, FLOAT_DTYPES
# import numpy as np
# import warnings
# from sklearn.base import BaseEstimator, TransformerMixin
# import scipy.sparse as sp
# from sklearn.feature_extraction.text import _document_frequency
# from sklearn.preprocessing import normalize

# class Bm25Vectorizer(CountVectorizer):
#     def __init__(self, input='content', encoding='utf-8',
#                  decode_error='strict', strip_accents=None, lowercase=True,
#                  preprocessor=None, tokenizer=None, analyzer='word',
#                  stop_words=None, token_pattern=r"\w+",
#                  ngram_range=(1, 1), max_df=1.0, min_df=1,
#                  max_features=None, vocabulary=None, binary=False,
#                  dtype=np.float64, norm='l2', use_idf=True, smooth_idf=True,
#                  sublinear_tf=False):

#         super(Bm25Vectorizer, self).__init__(
#             input=input, encoding=encoding, decode_error=decode_error,
#             strip_accents=strip_accents, lowercase=lowercase,
#             preprocessor=preprocessor, tokenizer=tokenizer, analyzer=analyzer,
#             stop_words=stop_words, token_pattern=token_pattern,
#             ngram_range=ngram_range, max_df=max_df, min_df=min_df,
#             max_features=max_features, vocabulary=vocabulary, binary=binary,
#             dtype=dtype)

#         self._tfidf = Bm25Transformer(norm=norm, use_idf=use_idf,
#                                        smooth_idf=smooth_idf,
#                                        sublinear_tf=sublinear_tf)

#     # Broadcast the TF-IDF parameters to the underlying transformer instance
#     # for easy grid search and repr

#     @property
#     def norm(self):
#         return self._tfidf.norm

#     @norm.setter
#     def norm(self, value):
#         self._tfidf.norm = value

#     @property
#     def use_idf(self):
#         return self._tfidf.use_idf

#     @use_idf.setter
#     def use_idf(self, value):
#         self._tfidf.use_idf = value

#     @property
#     def smooth_idf(self):
#         return self._tfidf.smooth_idf

#     @smooth_idf.setter
#     def smooth_idf(self, value):
#         self._tfidf.smooth_idf = value

#     @property
#     def sublinear_tf(self):
#         return self._tfidf.sublinear_tf

#     @sublinear_tf.setter
#     def sublinear_tf(self, value):
#         self._tfidf.sublinear_tf = value

#     @property
#     def idf_(self):
#         return self._tfidf.idf_

#     @idf_.setter
#     def idf_(self, value):
#         self._validate_vocabulary()
#         if hasattr(self, 'vocabulary_'):
#             if len(self.vocabulary_) != len(value):
#                 raise ValueError("idf length = %d must be equal "
#                                  "to vocabulary size = %d" %
#                                  (len(value), len(self.vocabulary)))
#         self._tfidf.idf_ = value

#     def _check_params(self):
#         if self.dtype not in FLOAT_DTYPES:
#             warnings.warn("Only {} 'dtype' should be used. {} 'dtype' will "
#                           "be converted to np.float64."
#                           .format(FLOAT_DTYPES, self.dtype),
#                           UserWarning)

#     def fit(self, raw_documents, y=None):
#         """Learn vocabulary and idf from training set.

#         Parameters
#         ----------
#         raw_documents : iterable
#             an iterable which yields either str, unicode or file objects

#         Returns
#         -------
#         self : TfidfVectorizer
#         """
#         self._check_params()
#         X = super(Bm25Vectorizer, self).fit_transform(raw_documents)
#         self._tfidf.fit(X)
#         return self

#     def fit_transform(self, raw_documents, y=None):
#         """Learn vocabulary and idf, return term-document matrix.

#         This is equivalent to fit followed by transform, but more efficiently
#         implemented.

#         Parameters
#         ----------
#         raw_documents : iterable
#             an iterable which yields either str, unicode or file objects

#         Returns
#         -------
#         X : sparse matrix, [n_samples, n_features]
#             Tf-idf-weighted document-term matrix.
#         """
#         self._check_params()
#         X = super(Bm25Vectorizer, self).fit_transform(raw_documents)
#         self._tfidf.fit(X)
#         # X is already a transformed view of raw_documents so
#         # we set copy to False
#         return self._tfidf.transform(X, copy=False)

#     def transform(self, raw_documents, copy=True):
#         """Transform documents to document-term matrix.

#         Uses the vocabulary and document frequencies (df) learned by fit (or
#         fit_transform).

#         Parameters
#         ----------
#         raw_documents : iterable
#             an iterable which yields either str, unicode or file objects

#         copy : boolean, default True
#             Whether to copy X and operate on the copy or perform in-place
#             operations.

#         Returns
#         -------
#         X : sparse matrix, [n_samples, n_features]
#             Tf-idf-weighted document-term matrix.
#         """
#         check_is_fitted(self, '_tfidf', 'The tfidf vector is not fitted')

#         X = super(Bm25Vectorizer, self).transform(raw_documents)
#         return self._tfidf.transform(X, copy=False)


# class Bm25Transformer(BaseEstimator, TransformerMixin):

#     def __init__(self,k=1.2,b=0.75, norm="l2", use_idf=True, smooth_idf=True,
#                  sublinear_tf=False):
#         self.k = k
#         self.b = b
#         ######### tfidf的代码###########
#         self.norm = norm
#         self.use_idf = use_idf
#         self.smooth_idf = smooth_idf
#         self.sublinear_tf = sublinear_tf

#     def fit(self, X, y=None):
#         """Learn the idf vector (global term weights)

#         Parameters
#         ----------
#         X : sparse matrix, [n_samples, n_features]
#             a matrix of term/token counts
#         """
#         X = X.toarray()
#         self.avdl = X.sum()/X.shape[0] #句子的平均长度
#         # print("原来的fit的数据：\n",X)

#         #计算每个词语的tf的值
#         self.tf = X.sum(0)/X.sum()  #[M] #M表示总词语的数量
#         self.tf = self.tf.reshape([1,self.tf.shape[0]]) #[1,M]
#         # print("tf\n",self.tf)
#         ######       原来tfidf的代码  ######

#         X = check_array(X, accept_sparse=('csr', 'csc'))
#         if not sp.issparse(X):
#             X = sp.csr_matrix(X)
#         dtype = X.dtype if X.dtype in FLOAT_DTYPES else np.float64

#         if self.use_idf:
#             n_samples, n_features = X.shape
#             df = _document_frequency(X).astype(dtype)

#             # perform idf smoothing if required
#             df += int(self.smooth_idf)
#             n_samples += int(self.smooth_idf)

#             # log+1 instead of log makes sure terms with zero idf don't get
#             # suppressed entirely.
#             idf = np.log(n_samples / df) + 1
#             self._idf_diag = sp.diags(idf, offsets=0,
#                                       shape=(n_features, n_features),
#                                       format='csr',
#                                       dtype=dtype)

#         return self

#     def transform(self, X, copy=True):
#         """Transform a count matrix to a tf or tf-idf representation

#         Parameters
#         ----------
#         X : sparse matrix, [n_samples, n_features]
#             a matrix of term/token counts

#         copy : boolean, default True
#             Whether to copy X and operate on the copy or perform in-place
#             operations.

#         Returns
#         -------
#         vectors : sparse matrix, [n_samples, n_features]
#         """
#         ########### 计算中间项  ###############
#         cur_tf = np.multiply(self.tf, X.toarray()) #[N,M] #N表示数据的条数，M表示总词语的数量
#         norm_lenght = 1 - self.b + self.b*(X.toarray().sum(-1)/self.avdl) #[N] #N表示数据的条数
#         norm_lenght = norm_lenght.reshape([norm_lenght.shape[0],1]) #[N,1]
#         X = (self.k+1)*cur_tf /(cur_tf +self.k*norm_lenght)
#         ############# 结算结束  ################

#         X = check_array(X, accept_sparse='csr', dtype=FLOAT_DTYPES, copy=copy)
#         if not sp.issparse(X):
#             X = sp.csr_matrix(X, dtype=np.float64)

#         n_samples, n_features = X.shape

#         if self.sublinear_tf:
#             np.log(X.data, X.data)
#             X.data += 1

#         if self.use_idf:
#             check_is_fitted(self, '_idf_diag', 'idf vector is not fitted')

#             expected_n_features = self._idf_diag.shape[0]
#             if n_features != expected_n_features:
#                 raise ValueError("Input has n_features=%d while the model"
#                                  " has been trained with n_features=%d" % (
#                                      n_features, expected_n_features))
#             # *= doesn't work

#             X = X * self._idf_diag

#         if self.norm:
#             X = normalize(X, norm=self.norm, copy=False)
#         return X

#     @property
#     def idf_(self):
#         # if _idf_diag is not set, this will raise an attribute error,
#         # which means hasattr(self, "idf_") is False
#         return np.ravel(self._idf_diag.sum(axis=0))

#     @idf_.setter
#     def idf_(self, value):
#         value = np.asarray(value, dtype=np.float64)
#         n_features = value.shape[0]
#         self._idf_diag = sp.spdiags(value, diags=0, m=n_features,
#                                     n=n_features, format='csr')

In [ ]:
# DATA_PATH = '../input/'
DATA_PATH = '../input/shopee-product-matching/'

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2, matplotlib.pyplot as plt
from tqdm import tqdm_notebook

import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors

def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score

In [ ]:
COMPUTE_CV = True

test = pd.read_csv(DATA_PATH + 'test.csv')
if len(test)>3: COMPUTE_CV = False
else: print('this submission notebook will compute CV score, but commit notebook will not')

# COMPUTE_CV = False

if COMPUTE_CV:
    train = pd.read_csv(DATA_PATH + 'train.csv')
    train['image'] = DATA_PATH + 'train_images/' + train['image']
    tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
    train['target'] = train.label_group.map(tmp)
    train_gf = cudf.read_csv(DATA_PATH + 'train.csv')
else:
    train = pd.read_csv(DATA_PATH + 'test.csv')
    train['image'] = DATA_PATH + 'test_images/' + train['image']
    train_gf = cudf.read_csv(DATA_PATH + 'test.csv')
    
print('train shape is', train.shape )
train.head()

# image hash

In [ ]:
tmp = train.groupby('image_phash').posting_id.agg('unique').to_dict()
train['oof_hash'] = train.image_phash.map(tmp)

In [ ]:
if COMPUTE_CV:
    train['f1'] = train.apply(getMetric('oof_hash'),axis=1)
    print('CV score for baseline =',train.f1.mean())

# image CNN

In [ ]:
from PIL import Image

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

class ShopeeImageDataset(Dataset):
    def __init__(self, img_path, transform):
        self.img_path = img_path
        self.transform = transform
        
    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')
        img = self.transform(img)
        return img
    
    def __len__(self):
        return len(self.img_path)

In [ ]:
imagedataset = ShopeeImageDataset(
    train['image'].values,
    transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]))
    
imageloader = torch.utils.data.DataLoader(
    imagedataset,
    batch_size=10, shuffle=False, num_workers=2
)

In [ ]:
class ShopeeImageEmbeddingNet(nn.Module):
    def __init__(self):
        super(ShopeeImageEmbeddingNet, self).__init__()
              
        model = models.resnet18(True)
        model.avgpool = nn.AdaptiveMaxPool2d(output_size=(1, 1))
        model = nn.Sequential(*list(model.children())[:-1])
        model.eval()
        self.model = model
        
    def forward(self, img):        
        out = self.model(img)
        return out

In [ ]:
!mkdir -p /root/.cache/torch/hub/checkpoints/
!cp ../input/pretrained-pytorch-models/resnet18-5c106cde.pth /root/.cache/torch/hub/checkpoints/

In [ ]:
DEVICE = 'cuda'

imgmodel = ShopeeImageEmbeddingNet()
imgmodel = imgmodel.to(DEVICE)

imagefeat = []
with torch.no_grad():
    for data in tqdm_notebook(imageloader):
        data = data.to(DEVICE)
        feat = imgmodel(data)
        feat = feat.reshape(feat.shape[0], feat.shape[1])
        feat = feat.data.cpu().numpy()
        
        imagefeat.append(feat)

In [ ]:
from sklearn.preprocessing import normalize

# l2 norm to kill all the sim in 0-1
imagefeat = np.vstack(imagefeat)
imagefeat = normalize(imagefeat)

In [ ]:
KNN = 50
if len(test)==3: KNN = 2
model = NearestNeighbors(n_neighbors=KNN)
model.fit(imagefeat)

In [ ]:
preds = []
CHUNK = 1024*4

imagefeat = cupy.array(imagefeat)

print('Finding similar images...')
CTS = len(imagefeat)//CHUNK
if len(imagefeat)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b, len(imagefeat))
    print('chunk',a,'to',b)
    
    distances = cupy.matmul(imagefeat, imagefeat[a:b].T).T
    # distances = np.dot(imagefeat[a:b,], imagefeat.T)
    
    for k in range(b-a):
        IDX = cupy.where(distances[k,]>0.95)[0]
        # IDX = np.where(distances[k,]>0.95)[0][:]
        o = train.iloc[cupy.asnumpy(IDX)].posting_id.values
        preds.append(o)
        
# del imagefeat, imgmodel

In [ ]:
train['oof_cnn'] = preds

if COMPUTE_CV:
    train['f1'] = train.apply(getMetric('oof_cnn'),axis=1)
    print('CV score for baseline =',train.f1.mean())

# title TFIDF

In [ ]:
# !pip install ../input/rank-bm25/rank_bm25-0.2.1-py3-none-any.whl
# # from gensim.summarization import bm25
# from rank_bm25 import BM25Okapi
# from nltk.tokenize import word_tokenize

In [ ]:

# train_title_token = train['title'].apply(lambda x: word_tokenize(x))
# train_array = []
# for i in range(0,34250):
#     train_array.append(train_title_token.iloc[i])

In [ ]:
# bm25 = BM25Okapi(train_title_token)
# bm25

In [ ]:
# train_title_token.iloc[0]

In [ ]:
# preds = []
# for i in train_title_token:
#     ids = bm25.get_scores(i)
#     ids = cupy.asarray(ids)
#     idx = cupy.where(ids>50)[0]
    
#     if not idx.any():
#         idx = cupy.where(ids==np.max(ids))[0]
        
#     o = train.iloc[cupy.asnumpy(idx)].posting_id.values
#     print(o)
#     preds.append(o)

In [ ]:
# preds = []
# for i in train_array:
#     ids = bm25.get_scores(i)
#     ids = np.asarray(ids)
#     idx = np.where(ids>50)[0]
    
#     if not idx.any():
#         idx = np.where(ids==np.max(ids))[0]
        
#     o = train.iloc[idx].posting_id.values
#     print(o)
#     preds.append(o)

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# model = TfidfVectorizer(stop_words=None, binary=True, max_features=25000)
# text_embeddings = model.fit_transform(train_gf.title).toarray()
# print('text embeddings shape',text_embeddings.shape)

In [ ]:
# train_array = train_gf.title.to_array()

In [ ]:
# bm_vec = Bm25Vectorizer(stop_words=None, binary=True, max_features=25000)
# # bm_vec.fit(train_array)
# text_embeddings = bm_vec.fit_transform(train_gf.title).toarray()
# print('text embeddings shape',text_embeddings.shape)

In [ ]:
# preds = []
# CHUNK = 1024*4

# print('Finding similar titles...')
# CTS = len(train)//CHUNK
# if len(train)%CHUNK!=0: CTS += 1
# for j in range( CTS ):
    
#     a = j*CHUNK
#     b = (j+1)*CHUNK
#     b = min(b,len(train))
#     print('chunk',a,'to',b)
    
#     # COSINE SIMILARITY DISTANCE
#     # cts = np.dot( text_embeddings, text_embeddings[a:b].T).T
#     cts = cupy.matmul(text_embeddings, text_embeddings[a:b].T).T
    
#     for k in range(b-a):
#         # IDX = np.where(cts[k,]>0.7)[0]
#         print(cts[k,])
#         IDX = cupy.where(cts[k,]>0.7)[0]
#         o = train.iloc[cupy.asnumpy(IDX)].posting_id.values
#         preds.append(o)
        
# del model, text_embeddings

In [ ]:
# preds

Word2evc

In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

train_title_token = train['title'].apply(lambda x: word_tokenize(x))
model = Word2Vec(sentences=train_title_token, vector_size=200, window=5, min_count=1, workers=4)
model.wv.save_word2vec_format("word2evc.txt",binary = "Ture/False")

In [ ]:
vectors = KeyedVectors.load_word2vec_format("word2evc.txt",binary = "Ture/False") # import the data file

In [ ]:
text_embeddings = []
for title in tqdm_notebook(train_title_token[:]):
    title_feat = []
    for word in title:
        if word in vectors:
            title_feat.append(vectors[word])
    
    if len(title_feat) == 0:
        title_feat = np.random.rand(200)
    else:
        # max-pooling
        # mean-pooling
        # IDF
        # SIF\
        
        title_feat = np.vstack(title_feat).max(0)
        
    text_embeddings.append(title_feat)
    # break

In [ ]:
from sklearn.preprocessing import normalize

# l2 norm to kill all the sim in 0-1
text_embeddings = np.vstack(text_embeddings)
text_embeddings = normalize(text_embeddings)

import torch
text_embeddings = torch.from_numpy(text_embeddings)
text_embeddings = text_embeddings.cuda()

In [ ]:
import cudf, cuml, cupy
text_embeddings = cupy.asarray(text_embeddings)

In [ ]:
preds = []
CHUNK = 1024*4


print('Finding similar images...')
CTS = len(text_embeddings)//CHUNK
if len(text_embeddings)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(train))
    print('chunk',a,'to',b)
    
    cts = cupy.matmul(text_embeddings, text_embeddings[a:b].T).T
    for k in range(b-a):
        IDX = cupy.where(cts[k,]>0.93)[0]
        o = train.iloc[cupy.asnumpy(IDX)].posting_id.values
        preds.append(o)
        
    del cts
    torch.cuda.empty_cache()

In [ ]:
train['oof_w2v'] = preds
# train['oof_text'] = preds

if COMPUTE_CV:
    train['f1'] = train.apply(getMetric('oof_w2v'),axis=1)
    print('CV score for baseline =',train.f1.mean())

In [ ]:
def combine_for_sub(row):
    x = np.concatenate([row.oof_w2v,row.oof_cnn, row.oof_hash])
    return ' '.join( np.unique(x) )

def combine_for_cv(row):
    x = np.concatenate([row.oof_w2v,row.oof_cnn, row.oof_hash])
    return np.unique(x)

In [ ]:
if COMPUTE_CV:
    tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
    train['target'] = train.label_group.map(tmp)
    train['oof'] = train.apply(combine_for_cv,axis=1)
    train['f1'] = train.apply(getMetric('oof'),axis=1)
    print('CV Score =', train.f1.mean() )

train['matches'] = train.apply(combine_for_sub,axis=1)

In [ ]:
train[['posting_id','matches']].to_csv('submission.csv',index=False)
sub = pd.read_csv('submission.csv')
sub.head()